In [1]:
import pandas as pd
import numpy as np
import sqlite3
conn = sqlite3.connect('consulates.sqlite')

In [2]:
df = pd.read_pickle("all_months.pkl")
DATA_START_DATE = df["Month"].min()
DATA_END_DATE = df["Month"].max()
COVID_START_DATE = pd.Timestamp("20200301")
NORMAL_MONTHS = pd.date_range(start=DATA_START_DATE, end=COVID_START_DATE, freq="MS", name="Month", closed="left")
COVID_MONTHS = pd.date_range(start=COVID_START_DATE, end=DATA_END_DATE, freq="MS", name="Month")
NORMAL_MONTHS, COVID_MONTHS

(DatetimeIndex(['2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01',
                '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
                '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
                '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01',
                '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
                '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01',
                '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
                '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01',
                '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01'],
               dtype='datetime64[ns]', name='Month', freq='MS'),
 DatetimeIndex(['2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01',
                '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01',
                '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01',
                '2021-03-01', '2021-04-01'],
               dtype='dat

In [9]:
df["Visa Class"] = df["Visa Class"].replace(
    to_replace={
        "B1": "B1/B2",
        "B2": "B1/B2",

        "BBBCC": "BBBCC/BBBCV",
        "BBBCV": "BBBCC/BBBCV",

        # conditionals based on https://travel.state.gov/content/dam/visas/Statistics/Immigrant-Statistics/MonthlyIVIssuances/Immigrant%20Visa%20Symbols.pdf
        "CR1": "CR1/IR1",
        "CR2": "CR2/IR2",
        "IR1": "CR1/IR1",
        "IR2": "CR2/IR2",

        "CX1": "CX1/FX1",
        "CX2": "CX2/FX2",
        "CX3": "CX3/FX3",
        "FX1": "CX1/FX1",
        "FX2": "CX2/FX2",
        "FX3": "CX3/FX3",
        "FX1": "CX1/FX1",

        "C21": "C21/F21",
        "C22": "C22/F22",
        "C23": "C23/F23",
        "C24": "C24/F24",
        "C25": "C25/F25",
        "F21": "C21/F21",
        "F22": "C22/F22",
        "F23": "C23/F23",
        "F24": "C24/F24",
        "F25": "C25/F25",

        "C31": "C31/F31",
        "C32": "C32/F32",
        "C33": "C33/F33",
        "F31": "C31/F31",
        "F32": "C32/F32",
        "F33": "C33/F33",

        "CW1": "CW1/IW1",
        "CW2": "CW2/IW2",
        "IW1": "CW1/IW1",
        "IW2": "CW2/IW2",

        "AM": "AM1",
        "BX": "BX1",
        "C2A": "C2",
        "CX": "C21/F21",
        "DV": "DV1",
        "E4": "SE1",
        "EW": "EW3",
        "F2A": "C21/F21",
        "F2B": "C24/F24",
        "F3": "C31/F31",
        "F4": "F41",
        "FX": "C21/F21",
        "I5": "I51",
        "IW": "CW1/IW1",
        "SD": "SD1",
        "SE": "SE1",
        "SI": "SI1",
        "SM": "SM1",
        "SQ": "SQ1",
        "SR": "SR1",
        "SU": "U1",
    }
)
df["Post"] = df["Post"].replace(
    to_replace={
        "AIT Taipei": "Taipei",
        "Dar Es Salaam": "Dar es Salaam",
        "Mumbai (Bombay)": "Mumbai",
        "Guatemala": "Guatemala City",
        "Panama": "Panama City",
        "Kuwait": "Kuwait City",
        "Port Au Prince": "Port-au-Prince",
        "Port Of Spain": "Port of Spain",
        "Rio De Janeiro": "Rio de Janeiro",
        "Tijuana Tpf": "Tijuana",
        "Chennai ( Madras)": "Chennai",
        "Chennai (Madras)": "Chennai",
    }
)
df = df.groupby(["Post", "Visa Class", "Month"], as_index=False, sort=False)[
    "Issuances"
].sum()
df


,Post,Visa Class,Month,Issuances
0,Monrovia,G4,2017-03-01,7
1,Brussels,E11,2017-03-01,1
2,Brussels,DV3,2017-03-01,1
3,Brussels,DV2,2017-03-01,1
4,Brussels,DV1,2017-03-01,2
...,...,...,...,...
265787,Kuala Lumpur,H1B,2021-04-01,18
265788,Kuala Lumpur,H3,2021-04-01,11
265789,Kuala Lumpur,H4,2021-04-01,17
265790,Kuala Lumpur,J1,2021-04-01,6


In [10]:
import re

def slugify(text: str) -> str:
    text = text.strip().lower()
    text = re.sub(r"[\s_]", "-", text)
    text = re.sub(r"[^\w-]", "", text)
    return text

df["Post Slug"] = df["Post"].apply(slugify)
df["Visa Class Slug"] = df["Visa Class"].apply(slugify)

post_slugs = pd.DataFrame(df[["Post Slug", "Post"]].drop_duplicates().set_index("Post Slug"))
post_slugs.to_sql("post_slugs", conn, if_exists="replace")

visa_slugs = pd.DataFrame(df[["Visa Class Slug", "Visa Class"]].drop_duplicates().set_index("Visa Class Slug"))

visa_descriptions_sheet_url = "https://docs.google.com/spreadsheets/d/1GLtqb3PVXFriG7lanunfYJrpUE7_e0_L-PEN2spBQbY/export?format=csv"

visa_descriptions = pd.read_csv(visa_descriptions_sheet_url).set_index("Visa Class Slug")

visa_slugs = visa_slugs.join(visa_descriptions, on="Visa Class Slug")
visa_slugs.to_sql("visa_slugs", conn, if_exists="replace")

/Users/underyx/.local/pipx/venvs/jupyter/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [11]:
def get_baseline(df: pd.DataFrame) -> pd.DataFrame:
    df = df.set_index("Month")
    return df[["Issuances"]].reindex(index=NORMAL_MONTHS, fill_value=0).mean()

baselines = df.groupby(["Post Slug", "Visa Class Slug"], sort=False).apply(get_baseline)
baselines

Issuances
Post Slug Visa Class Slug           
monrovia  g4                2.638889
brussels  e11               0.222222
          dv3               0.777778
          dv2               0.666667
          dv1               1.805556
...                              ...
tbilisi   sd1               0.000000
monterrey e3r               0.000000
jerusalem a1                0.000000
islamabad h1b1              0.000000
la-paz    g3                0.000000

[14141 rows x 1 columns]

In [12]:
df.loc[df["Post Slug"] == "guayaquil"].loc[df["Visa Class Slug"] == "f3"]

,Post,Visa Class,Month,Issuances,Post Slug,Visa Class Slug


In [13]:
baselines.loc[("budapest", "cr1ir1")]

Issuances    3.361111
Name: (budapest, cr1ir1), dtype: float64

In [14]:
baselines.to_pickle("baselines.pkl")
baselines.to_sql("baselines", conn, if_exists="replace")

/Users/underyx/.local/pipx/venvs/jupyter/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [15]:
def get_backlog(df: pd.DataFrame) -> pd.DataFrame:
    df = df.reset_index()
    key = tuple(df.loc[0, ["Post Slug", "Visa Class Slug"]])
    baseline = baselines.loc[key][0]

    df = df.set_index("Month")

    normal_months = df[["Issuances"]].reindex(index=NORMAL_MONTHS, fill_value=0)

    covid_months = df[["Issuances"]].reindex(index=COVID_MONTHS, fill_value=0)
    covid_months["Actual Progress"] = covid_months["Issuances"].cumsum()
    covid_months["Expected Progress"] = np.linspace(baseline, baseline * len(covid_months), num=len(covid_months))
    covid_months["Backlog"] = covid_months["Actual Progress"] - covid_months["Expected Progress"]
    covid_months["Months Ahead"] = covid_months["Backlog"] / baseline
    covid_months = covid_months.drop(columns=["Actual Progress", "Expected Progress"])

    return pd.concat([normal_months, covid_months])[["Issuances", "Backlog", "Months Ahead"]]

backlogs = df.groupby(["Post Slug", "Visa Class Slug"], sort=False).apply(get_backlog)

In [ ]:
backlogs.to_pickle("backlogs.pkl")

In [ ]:
backlogs.to_sql("backlogs", conn, if_exists="replace")